In [3]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels.api as sm
from random import random
from sklearn.metrics import mean_squared_error
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from tqdm import tqdm_notebook
import itertools

In [4]:
df = pd.read_csv('climate_test.csv', delimiter=';').drop('ID', axis=1)
pred_res = list()

In [5]:
ts = df.iloc[:,0]
ts_val = ts[792:828]
ts_train = ts[0:792]

Будем искать параметры модели SARIMAX по сетке, все паметры от 0 до 2. Выбираем лучший вариант и прогнозируем для kaggle.

In [4]:
p = range(0, 3)
d = range(0, 3)
q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [5]:
for i in tqdm_notebook(range(10)):
    ts = df.iloc[:, i]
    ts_val = ts[792:828]
    ts_train = ts[0:792]
    pdq_best = (-1,-1,-1)
    PDQ_best = (-1,-1,-1)
    rmse_best = 10000
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(ts_train,
                order=param,
                seasonal_order=param_seasonal,
                enforce_stationarity=False,
                enforce_invertibility=False)
                results = mod.fit(disp=-1)
                pred = results.predict(start=792, end=827, dynamic=True)
                MSE = mean_squared_error(ts_val, pred)
                rmse = math.sqrt(MSE)
                if (rmse < rmse_best):
                    rmse_best = rmse
                    pdq_best = param
                    PDQ_best = param_seasonal
            except:
                continue
    mod = sm.tsa.statespace.SARIMAX(ts,
                order=pdq_best,
                seasonal_order=PDQ_best,
                enforce_stationarity=False,
                enforce_invertibility=False)
    results = mod.fit(disp=-1)
    pred_res = pred_res + results.predict(start=828, end=863, dynamic=True).to_list()

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
df1 = pd.read_csv('climate_for_bw_onecol.csv', delimiter=',').drop('Predicted', axis=1)

In [7]:
d1 = pd.DataFrame({'Id' : df1.values.flatten(), 'Predicted': pred_res})
d1.to_csv('test.csv', index=False)